<a href="https://colab.research.google.com/github/jetsonmom/roboflow-test/blob/main/roboflow_code_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install Roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="XXXXXXXX")
project = rf.workspace().project("lane_jetsonmom") #model end point
model = project.version(3).model

# infer on a local image
print(model.predict("/content/sample_data/lane1.jpg", confidence=40, overlap=30).json())

# visualize your prediction
#model.predict("your_image.jpg", confidence=40, overlap=30).save("prediction.jpg")

# infer on an image hosted elsewhere
#print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())

In [ ]:
from roboflow import Roboflow
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Your API key and model loading
rf = Roboflow(api_key="nHzqEuoiMsMPExQOq3qo")
project = rf.workspace().project("lane_jetsonmom") #model end point
model = project.version(3).model

# Load and display the image
img_path = "/content/sample_data/lane1.jpg"  # Adjust the path to where your image is
img = mpimg.imread(img_path)
plt.imshow(img)
plt.show()

# Make predictions
prediction = model.predict(img_path, confidence=10, overlap=30)
print(prediction.json())  # Display the JSON output

# Save and display the prediction visualization
prediction_image_path = "prediction.jpg"
model.predict(img_path, confidence=15, overlap=30).save(prediction_image_path)
pred_img = mpimg.imread(prediction_image_path)
plt.imshow(pred_img)
plt.show()


In [ ]:
import cv2
from roboflow import Roboflow
import matplotlib.pyplot as plt

# Roboflow API 설정
rf = Roboflow(api_key="XXXXXXXX")
project = rf.workspace().project("lane_jetsonmom") #model end point
model = project.version(3).model

# 비디오 파일 열기
cap = cv2.VideoCapture('/content/sample_data/video.mp4')

# 비디오가 열리지 않았다면 종료
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# 프레임별로 읽기
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV 이미지를 RGB로 변환 (출력용)
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 파일로 임시 저장
    cv2.imwrite('temp_frame.jpg', frame)

    # 모델 예측
    prediction = model.predict('temp_frame.jpg', confidence=10, overlap=30)
    print(prediction.json())


In [ ]:
import cv2
from roboflow import Roboflow
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Roboflow API 설정
rf = Roboflow(api_key="nHzqEuoiMsMPExQOq3qo")
project = rf.workspace().project("lane_jetsonmom")
model = project.version(3).model

# 비디오 파일 열기
cap = cv2.VideoCapture('/content/sample_data/video.mp4')

# 비디오가 열리지 않았다면 종료
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# 프레임별로 읽기
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 파일로 임시 저장
    cv2.imwrite('temp_frame.jpg', frame)

    # 모델 예측
    prediction = model.predict('temp_frame.jpg', confidence=10, overlap=30).json()

    # 이미지를 matplotlib로 로드
    img = Image.open('temp_frame.jpg')
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # 바운딩 박스 그리기
    for obj in prediction['predictions']:
        rect = patches.Rectangle((obj['x'], obj['y']), obj['width'], obj['height'], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    plt.show()


In [ ]:
import cv2
from roboflow import Roboflow
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import io

# Roboflow API 설정
rf = Roboflow(api_key="nHzqEuoiMsMPExQOq3qo")
project = rf.workspace().project("lane_jetsonmom")
model = project.version(3).model

# 비디오 파일 열기
cap = cv2.VideoCapture('/content/sample_data/video.mp4')

# 비디오가 열리지 않았다면 종료
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# 프레임별로 읽기
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임을 PIL 이미지로 변환
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(frame_rgb)

    # 메모리에 이미지를 임시 저장
    buf = io.BytesIO()
    img_pil.save(buf, format='JPEG')
    byte_im = buf.getvalue()

    # 모델 예측
    prediction = model.predict(byte_im, confidence=10, overlap=30).json()

    # 이미지를 matplotlib로 로드
    fig, ax = plt.subplots(1)
    ax.imshow(img_pil)

    # 바운딩 박스 그리기
    for obj in prediction['predictions']:
        rect = patches.Rectangle((obj['x'], obj['y']), obj['width'], obj['height'], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    plt.show()
    plt.close()  # 다음 프레임을 위해 그래프를 닫음

# 작업 완료 후 자원 해제
cap.release()


In [ ]:
import cv2
from roboflow import Roboflow

# Roboflow API 설정
rf = Roboflow(api_key="nHzqEuoiMsMPExQOq3qo")
project = rf.workspace().project("lane_jetsonmom")
model = project.version(3).model

# 비디오 파일 열기
cap = cv2.VideoCapture('/content/sample_data/video.mp4')
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (int(cap.get(3)), int(cap.get(4))))

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    # 파일로 임시 저장
    cv2.imwrite('temp_frame.jpg', frame)

    # 모델 예측
    prediction = model.predict('temp_frame.jpg', confidence=10, overlap=30).json()

    # 바운딩 박스 그리기
    for obj in prediction['predictions']:
        cv2.rectangle(frame, (int(obj['x']), int(obj['y'])), (int(obj['x'] + obj['width']), int(obj['y'] + obj['height'])), (0, 255, 0), 2)

    # 변형된 프레임을 비디오로 저장
    out.write(frame)

cap.release()
out.release()

# 비디오 파일을 다운로드
from google.colab import files
files.download('output.mp4')
